<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/NoumetodeANON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langdetect deep_translator tqdm transformers torch

import sqlite3
import pandas as pd
import re
import time
from langdetect import detect, DetectorFactory, LangDetectException
from deep_translator import GoogleTranslator
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from transformers import pipeline

# Configuración inicial
DetectorFactory.seed = 0

# Cargar modelos
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")
name_generator = pipeline("text-generation", model="PlanTL-GOB-ES/gpt2-large-bne", device=0)  # Usar GPU si está disponible

def detectar_genero(nombre):
    """Detecta el género usando el modelo de lenguaje"""
    prompt = f"""Clasifica el género del siguiente nombre español. Responde solo con una palabra: masculino, femenino o neutral.
Nombre: {nombre.split()[0]}
Género:"""

    response = name_generator(
        prompt,
        max_length=50,
        num_return_sequences=1,
        truncation=True,
        temperature=0.01  # Bajar temperatura para respuestas más determinísticas
    )[0]['generated_text']

    # Extraer la respuesta
    response = response.replace(prompt, "").strip().lower()
    if 'masculino' in response: return "male"
    if 'femenino' in response: return "female"
    return "neutral"

def generar_nombre(original_name):
    """Genera un nombre con el mismo número de palabras y género usando LM"""
    palabras = original_name.split()
    num_palabras = len(palabras)
    genero = detectar_genero(original_name)

    # Definir plantilla según número de palabras
    plantillas = {
        1: "Genera un nombre de pila {gender} español. Solo el nombre sin apellidos.",
        2: "Genera un nombre completo {gender} español con 1 nombre y 1 apellido.",
        3: "Genera un nombre completo {gender} español con 1 nombre y 2 apellidos."
    }

    prompt = plantillas.get(num_palabras, plantillas[3]).format(
        gender="masculino" if genero == "male" else "femenino"
    )

    for _ in range(3):  # Reintentos en caso de error
        response = name_generator(
            prompt,
            max_length=30,
            num_return_sequences=1,
            truncation=True,
            temperature=0.7
        )[0]['generated_text']

        # Limpiar respuesta
        generated = re.sub(r"[^a-zA-ZáéíóúñÁÉÍÓÚÑ\s]", "", response.split("\n")[0]).strip()

        if len(generated.split()) == num_palabras:
            return generated.title()

    # Fallback para errores
    return "Juan Martínez García" if num_palabras >= 3 else "Ana López"

def detectar_entidades(texto):
    """Detecta entidades PER usando BERT"""
    resultados = ner_model(texto)
    entidades = []

    for ent in resultados:
        if ent['entity_group'] == 'PER':
            start = ent['start']
            end = ent['end']
            original = texto[start:end]

            # Ajustar límites de la entidad
            while start > 0 and texto[start-1] not in (' ', '\n', '\t'):
                start -= 1
            while end < len(texto) and texto[end] not in (' ', '\n', '\t'):
                end += 1

            entidades.append((start, end, original.strip()))

    return entidades

def traducir_y_anonimizar(texto):
    """Proceso completo de anonimización"""
    if not texto.strip():
        return texto

    # Traducción a español
    try:
        if len(texto) > 3 and detect(texto) != "es":
            texto = GoogleTranslator(source="auto", target="es").translate(texto)
    except LangDetectException:
        pass

    # Detección y reemplazo de entidades
    entidades = detectar_entidades(texto)
    replacements = []

    for start, end, original in entidades:
        fake_name = generar_nombre(original)
        replacements.append((start, end, fake_name))

    # Aplicar reemplazos en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    return texto

def procesar_fila(row):
    """Procesa una fila aplicando traducción y anonimización."""
    row["body"] = traducir_y_anonimizar(row["body"])
    return row

# Resto del código de procesamiento de base de datos (igual que el original)
# ... (el código de conexión a BD y procesamiento paralelo se mantiene igual)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.5 MB/s eta 0:00:00
 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Device set to use cpu


In [3]:
# Conectar a la base de datos
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    # Crear la tabla si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS mi_tabla (
            id INTEGER PRIMARY KEY,
            body TEXT,
            secret TEXT,
            direction TEXT,
            createdAt TEXT,
            OpenchannelAccountId INTEGER,
            OpenchannelInteractionId INTEGER,
            UserId INTEGER,
            ContactId INTEGER,
            AttachmentId INTEGER,
            sentBy TEXT
        );
    """)
    conn.commit()

    # Cargar datos desde Excel (solo si es necesario)
    df = pd.read_excel("MOSTRA_1.xlsx")
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer solo columnas relevantes
    df_body = df[["id", "body", "direction","createdAt","UserId", "ContactId"]].copy()

    # Usar tqdm para mostrar progreso
    start_time = time.time()
    with Pool(cpu_count()) as pool:
        result = list(tqdm(pool.imap(procesar_fila, df_body.to_dict(orient="records")), total=len(df_body)))

    # Convertir la lista de diccionarios de vuelta a DataFrame
    df_body = pd.DataFrame(result)

    # Actualizar base de datos en un solo paso eficiente
    df_body.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    elapsed_time = time.time() - start_time
    print(f"Procesamiento completado en {elapsed_time:.2f} segundos.")

  0%|          | 0/6943 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
 15%|█▍        | 1023/6943 [06:38<38:27,  2.57it/s]  


KeyboardInterrupt: 

In [4]:
import sqlite3
import pandas as pd
import re
import time
from langdetect import detect, DetectorFactory, LangDetectException
from deep_translator import GoogleTranslator
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from transformers import pipeline

# Configuración inicial
DetectorFactory.seed = 0

# Cargar modelos
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")
name_generator = pipeline("text-generation", model="PlanTL-GOB-ES/gpt2-large-bne", device=0)  # Usar GPU si está disponible

def detectar_genero(nombre):
    """Detecta el género usando el modelo de lenguaje"""
    prompt = f"""Clasifica el género del siguiente nombre español. Responde solo con una palabra: masculino, femenino o neutral.
Nombre: {nombre.split()[0]}
Género:"""

    response = name_generator(
        prompt,
        max_length=50,
        num_return_sequences=1,
        truncation=True,
        temperature=0.01  # Bajar temperatura para respuestas más determinísticas
    )[0]['generated_text']

    # Extraer la respuesta
    response = response.replace(prompt, "").strip().lower()
    if 'masculino' in response: return "male"
    if 'femenino' in response: return "female"
    return "neutral"

def generar_nombre(original_name):
    """Genera un nombre con el mismo número de palabras y género usando LM"""
    palabras = original_name.split()
    num_palabras = len(palabras)
    genero = detectar_genero(original_name)

    # Definir plantilla según número de palabras
    plantillas = {
        1: "Genera un nombre de pila {gender} español. Solo el nombre sin apellidos.",
        2: "Genera un nombre completo {gender} español con 1 nombre y 1 apellido.",
        3: "Genera un nombre completo {gender} español con 1 nombre y 2 apellidos."
    }

    prompt = plantillas.get(num_palabras, plantillas[3]).format(
        gender="masculino" if genero == "male" else "femenino"
    )

    for _ in range(3):  # Reintentos en caso de error
        response = name_generator(
            prompt,
            max_length=30,
            num_return_sequences=1,
            truncation=True,
            temperature=0.7
        )[0]['generated_text']

        # Limpiar respuesta
        generated = re.sub(r"[^a-zA-ZáéíóúñÁÉÍÓÚÑ\s]", "", response.split("\n")[0]).strip()

        if len(generated.split()) == num_palabras:
            return generated.title()

    # Fallback para errores
    return "Juan Martínez García" if num_palabras >= 3 else "Ana López"

def detectar_entidades(texto):
    """Detecta entidades PER usando BERT"""
    resultados = ner_model(texto)
    entidades = []

    for ent in resultados:
        if ent['entity_group'] == 'PER':
            start = ent['start']
            end = ent['end']
            original = texto[start:end]

            # Ajustar límites de la entidad
            while start > 0 and texto[start-1] not in (' ', '\n', '\t'):
                start -= 1
            while end < len(texto) and texto[end] not in (' ', '\n', '\t'):
                end += 1

            entidades.append((start, end, original.strip()))

    return entidades

def traducir_y_anonimizar(texto):
    """Proceso completo de anonimización"""
    if not texto.strip():
        return texto

    # Traducción a español
    try:
        if len(texto) > 3 and detect(texto) != "es":
            texto = GoogleTranslator(source="auto", target="es").translate(texto)
    except LangDetectException:
        pass

    # Detección y reemplazo de entidades
    entidades = detectar_entidades(texto)
    replacements = []

    for start, end, original in entidades:
        fake_name = generar_nombre(original)
        replacements.append((start, end, fake_name))

    # Aplicar reemplazos en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    return texto

def procesar_fila(row):
    """Procesa una fila aplicando traducción y anonimización."""
    row["body"] = traducir_y_anonimizar(row["body"])
    return row
    # Conectar a la base de datos
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    # Crear la tabla si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS mi_tabla (
            id INTEGER PRIMARY KEY,
            body TEXT,
            secret TEXT,
            direction TEXT,
            createdAt TEXT,
            OpenchannelAccountId INTEGER,
            OpenchannelInteractionId INTEGER,
            UserId INTEGER,
            ContactId INTEGER,
            AttachmentId INTEGER,
            sentBy TEXT
        );
    """)
    conn.commit()

    # Cargar datos desde Excel (solo si es necesario)
    df = pd.read_excel("MOSTRA_1.xlsx")
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer solo columnas relevantes
    df_body = df[["id", "body", "direction","createdAt","UserId", "ContactId"]].copy()

    # Usar tqdm para mostrar progreso
    start_time = time.time()
    with Pool(cpu_count()) as pool:
        result = list(tqdm(pool.imap(procesar_fila, df_body.to_dict(orient="records")), total=len(df_body)))

    # Convertir la lista de diccionarios de vuelta a DataFrame
    df_body = pd.DataFrame(result)

    # Actualizar base de datos en un solo paso eficiente
    df_body.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    elapsed_time = time.time() - start_time
    print(f"Procesamiento completado en {elapsed_time:.2f} segundos.")

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu
  0%|          | 0/6943 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no pred

KeyboardInterrupt: 